# MOF ChemUnity Property Extraction

This notebook demonstrates how the property extraction in MOF ChemUnity is used. You need to have the MOF name that you want to extract properties for which is obtained from the Matching workflow.

In [38]:
from src.MOF_ChemUnity.Agents.ExtractionAgent import ExtractionAgent
from src.MOF_ChemUnity.utils.DataPrep import Data_Prep
from src.MOF_ChemUnity.Extraction_Prompts import VERIFICATION, RECHECK, EXTRACTION
from src.MOF_ChemUnity.Water_Stability_Prompts import WATER_STABILITY, RULES_WATER_STABILITY, VERF_RULES_WATER_STABILITY, WATER_STABILITY_RE

### Preparation of MOF Names from Matching CSV

we need to read the matching csv file and extract the file names from within that.

In [39]:
import pandas as pd
import glob
import os

In [40]:
mof_names_df = pd.read_csv("./tests/water_stability_benchmark/case-study-3-ground-truth.csv")
mof_names_df.head()

,Reference,DOI,MOF contained,True Water Stability,Justification Sentence,Unnamed: 5
0,1,10.1038/s41586-019-1798-7,Al-PMOF/m8o66,Stable,The capture capacity of Al-PMOF for a mixture ...,NaN
1,1,10.1038/s41586-019-1798-7,Al-PyrMOF/m8o67,Stable,The capture capacity of Al-PyrMOF for a mixtur...,NaN
2,1,10.1038/s41586-019-1798-7,UiO-66-NH2,Not provided,Not provided,NaN
3,1,10.1038/s41586-019-1798-7,m8o71,Unstable,"Conversely, m8o71 completely loses its CO2 cap...",NaN
4,2,10.1039/c0dt00999g,[Zn4(dmf)(ur)2(ndc)4],Stable,The high stability of the guest-free metal–org...,NaN


### Markdown files setup

In [41]:
input_folder = "./tests/water_stability_benchmark/markdown"


files = glob.glob(input_folder+"/*/*.md")

In [42]:
mof_names_df["File"] = [input_folder+f"/{i}/{i}.md" for i in list(mof_names_df["Reference"])]
mof_names_df.head()

,Reference,DOI,MOF contained,True Water Stability,Justification Sentence,Unnamed: 5,File
0,1,10.1038/s41586-019-1798-7,Al-PMOF/m8o66,Stable,The capture capacity of Al-PMOF for a mixture ...,NaN,./tests/water_stability_benchmark/markdown/1/1.md
1,1,10.1038/s41586-019-1798-7,Al-PyrMOF/m8o67,Stable,The capture capacity of Al-PyrMOF for a mixtur...,NaN,./tests/water_stability_benchmark/markdown/1/1.md
2,1,10.1038/s41586-019-1798-7,UiO-66-NH2,Not provided,Not provided,NaN,./tests/water_stability_benchmark/markdown/1/1.md
3,1,10.1038/s41586-019-1798-7,m8o71,Unstable,"Conversely, m8o71 completely loses its CO2 cap...",NaN,./tests/water_stability_benchmark/markdown/1/1.md
4,2,10.1039/c0dt00999g,[Zn4(dmf)(ur)2(ndc)4],Stable,The high stability of the guest-free metal–org...,NaN,./tests/water_stability_benchmark/markdown/2/2.md


### Running the Extraction Loop for General Property Extraction + CoV

In [43]:
with open(".apikeys", 'r') as f:
    os.environ["OPENAI_API_KEY"] = f.read()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", temperature=0.1)
parser_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [44]:
result = {}
result["MOF Name"] = []
result["Ref Code"] = []
result["Property"] = []
result["Value"] = []
result["Units"] = []
result["Condition"] = []
result["Summary"] = []
result["Reference"] = []

In [45]:
filtered_result = {}
filtered_result["MOF Name"] = []
filtered_result["Ref Code"] = []
filtered_result["Property"] = []
filtered_result["Value"] = []
filtered_result["Units"] = []
filtered_result["Condition"] = []
filtered_result["Summary"] = []
filtered_result["Reference"] = []

In [46]:
ws_result = {}
ws_result["MOF Name"] = []
ws_result["Ref Code"] = []
ws_result["Property"] = []
ws_result["Value"] = []
ws_result["Units"] = []
ws_result["Condition"] = []
ws_result["Summary"] = []
ws_result["Reference"] = []

In [47]:
WS_READ = WATER_STABILITY.replace("{RULES}", RULES_WATER_STABILITY)
WS_CHECK = VERIFICATION.replace("{VERF_RULES}", VERF_RULES_WATER_STABILITY)
WS_RECHECK = RECHECK.replace("{RECHECK_INSTRUCTIONS}", WATER_STABILITY_RE.replace("{RULES}", RULES_WATER_STABILITY))

sp_dict = {"read_prompts": [WS_READ], "verification_prompts": [WS_CHECK], "recheck_prompts": [WS_RECHECK]}

In [48]:
agent = ExtractionAgent(llm=llm, parser_llm=parser_llm)

In [ ]:
for i in range(len(mof_names_df)):

    mof = mof_names_df.iloc[i]["MOF contained"]
    refcode = "HELLOW"
    reference = mof_names_df.iloc[i]["DOI"]
    
    _, response = agent.agent_response(mof, mof_names_df.iloc[i]["File"],
                                    EXTRACTION, ["Water Stability"], sp_dict, CoV=True, skip_general=True, fuzz_threshold=85, store_vs=True)
    
    # general_extraction = response

    # filtered = general_extraction[0]
    # all_props = general_extraction[1]

    # print(filtered)
    # print(all_props)

    # for j in filtered:
    #     filtered_result["MOF Name"].append(mof)
    #     filtered_result["Ref Code"].append(refcode)
    #     filtered_result["Reference"].append(reference)
    #     filtered_result["Property"].append(j.name)
    #     filtered_result["Units"].append(j.units)
    #     filtered_result["Value"].append(j.value)
    #     filtered_result["Condition"].append(j.condition)
    #     filtered_result["Summary"].append(j.summary)
    # for j in all_props.properties:
    #     result["MOF Name"].append(mof)
    #     result["Ref Code"].append(refcode)
    #     result["Reference"].append(reference)
    #     result["Property"].append(j.name)
    #     result["Units"].append(j.units)
    #     result["Value"].append(j.value)
    #     result["Condition"].append(j.condition)
    #     result["Summary"].append(j.summary)
    
    specific_extraction = response

    ws = specific_extraction[0]

    for j in ws:
        ws_result["MOF Name"].append(mof)
        ws_result["Ref Code"].append(refcode)
        ws_result["Reference"].append(reference)
        ws_result["Property"].append(j.name)
        ws_result["Units"].append(j.units)
        ws_result["Value"].append(j.value)
        ws_result["Condition"].append(j.condition)
        ws_result["Summary"].append(j.summary)



# all_props = pd.DataFrame(result)
# filtered = pd.DataFrame(filtered_result)
ws = pd.DataFrame(ws_result)
    

Action: reading the document
finding all properties of name 1: [Cu3(TP)4(N3)2(DMF)2]·3C6H12 ---name 2: 1a

Result: 
1.  
   - Property Name: Space Group
   - Property Value: P21/n
   - Value Units: N/A
   - Conditions: N/A
   - Summary: "Single-crystal X-ray analysis reveals that 1a still maintains the original monoclinic space group P21/n (Supporting Information Table S1)."

2.  
   - Property Name: Cell Volume Expansion
   - Property Value: 3.74
   - Value Units: %
   - Conditions: N/A
   - Summary: "There is only a little expansion of a, b, and c axes (Δa = 0.96%, Δb = 1.89%, and Δc = 1.02%), which consequently results in a small expansion of the cell volume (ΔV = 3.74%) (Supporting Information Table S11)."

3.  
   - Property Name: Square Window Size
   - Property Value: 13.261 x 21.697
   - Value Units: Å
   - Conditions: N/A
   - Summary: "However, the sizes of the square windows have obvious variations from ca.13.135 Å × 21.390 Å in 1 to ca.13.261 Å × 21.697 Å in 1a (Supporting 

ValueError: If using all scalar values, you must pass an index

In [ ]:
ws.to_csv("/mnt/c/Users/Amro/Desktop/ws.csv")